### Info
based on notebook4

compare qkeras and qonnx predictions (without using hls4ml)\
strange thing on github: pred_qonnx[0]: [[4.4136963]]; then: 4.4375

### Load packages and the model

In [ ]:
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

### Copy weights

In [ ]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

In [ ]:
#Quick setup (load qonnx model)
import tensorflow as tf
import numpy as np
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits

import qonnx.core.onnx_exec as oxe
from qonnx.converters import from_keras
from qonnx.core.modelwrapper import ModelWrapper

qonnx_model = ModelWrapper("mod_model.onnx")

2025-09-27 14:53:58.724508: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-27 14:53:58.767124: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-27 14:53:58.767266: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-27 14:53:58.767331: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-27 14:53:58.778524: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-27 14:53:58.779561: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### QONNX model

In [ ]:
import qonnx.core.onnx_exec as oxe
from qonnx.converters import from_keras
from qonnx.core.modelwrapper import ModelWrapper
#from qonnx.transformation.infer_shapes import InferShapes

qonnx_model, external_storage = from_keras(model)
qonnx_model = ModelWrapper(qonnx_model)

In [7]:
node_names = []
for node in qonnx_model.graph.node:
    node_names.append(node.name)

print(node_names)

['Quant_0', 'Quant_1', 'Quant_2', 'Quant_3', 'Quant_4', 'Quant_5', 'Quant_6', 'Quant_7', 'Quant_8', 'Quant_9', 'Quant_10', 'Quant_11', 'Quant_12', 'Quant_13', 'Quant_14', 'Quant_15', 'MatMul_0', 'Add_0', 'Quant_16', 'Relu_0', 'Quant_17', 'MatMul_1', 'Add_1', 'Quant_18', 'Relu_1', 'Quant_19', 'MatMul_2', 'Add_2', 'Quant_20', 'Relu_2', 'Quant_21', 'MatMul_3', 'Add_3', 'Quant_22', 'Relu_3', 'Quant_23', 'MatMul_4', 'Add_4', 'Quant_24', 'MatMul_5', 'Add_5', 'Quant_25', 'Relu_4', 'Quant_26', 'MatMul_6', 'Add_6', 'Quant_27', 'Relu_5', 'Quant_28', 'MatMul_7', 'Add_7', 'Sub_0', 'Unsqueeze_0', 'Unsqueeze_1', 'MatMul_8', 'Squeeze_0', 'Identity_0', 'Identity_1']


### Check the input parameters of the Quant nodes (adjust the inputs and attributes)
0.125 = (0.001)_2 \
0.0625 = (0.0001)_2 \
0.0009765625 = (0.0000000001)_2

In [8]:
qnodes = list(filter(lambda x: x.op_type == "Quant", qonnx_model.graph.node))

# qnode consumers and param inputs
for qnode in qnodes[16:]:
    print("Quant node name: ",qnode.name)
    print("Consumer name: ",qonnx_model.find_consumer(qnode.output[0]).name)
    for inp in qnode.input[1:]:
        print(inp, qonnx_model.get_initializer(inp))
    print("--------------------")
# Only the Quant_19 node has two consumers (value after 'MatMul_2', later also 'Sub_0')

Quant node name:  Quant_16
Consumer name:  Relu_0
Quant_16_param0 0.0625
Quant_16_param1 0
Quant_16_param2 10
--------------------
Quant node name:  Quant_17
Consumer name:  MatMul_1
Quant_17_param0 0.0625
Quant_17_param1 0
Quant_17_param2 10
--------------------
Quant node name:  Quant_18
Consumer name:  Relu_1
Quant_18_param0 0.0625
Quant_18_param1 0
Quant_18_param2 10
--------------------
Quant node name:  Quant_19
Consumer name:  MatMul_2
Quant_19_param0 0.0625
Quant_19_param1 0
Quant_19_param2 10
--------------------
Quant node name:  Quant_20
Consumer name:  Relu_2
Quant_20_param0 0.0625
Quant_20_param1 0
Quant_20_param2 10
--------------------
Quant node name:  Quant_21
Consumer name:  MatMul_3
Quant_21_param0 0.0625
Quant_21_param1 0
Quant_21_param2 10
--------------------
Quant node name:  Quant_22
Consumer name:  Relu_3
Quant_22_param0 0.0625
Quant_22_param1 0
Quant_22_param2 10
--------------------
Quant node name:  Quant_23
Consumer name:  MatMul_4
Quant_23_param0 0.0625
Qu

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/qonnx/core/modelwrapper.py:417: UserWarning: find_consumer: found multiple consumers, returning first one
  warnings.warn("find_consumer: found multiple consumers, returning first one")


In [45]:
qonnx_model.set_initializer("Quant_16_param0",np.array(0.0009765625))

In [29]:
# qnode attributes
for qnode in qnodes[16:17]:
    print("Quant node name: ",qnode.name)
    print(qnode.attribute)
    print("---------------")

Quant node name:  Quant_16
[name: "narrow"
i: 0
type: INT
, name: "rounding_mode"
s: "ROUND"
type: STRING
, name: "signed"
i: 1
type: INT
]
---------------


In [26]:
from qonnx.custom_op.base import CustomOp
from qonnx.custom_op.registry import getCustomOp

node_op = getCustomOp(qonnx_model.get_finn_nodes()[16])

In [28]:
node_op.set_nodeattr("signed",1)

### Conclusions
In hls input_1 (global input) should have precision fixed<14,7> (now fixed<18,8>)\
Relu should take input with precision like the accumulator: fixed<18,8>\
Should check the "signed" attribute of Quant nodes\
missing quantizers before 'Sub_0' and later on

Quant node is not present between each add and relu within the Dense layers.\
These quant nodes should correspond to accumulators (?)

In [ ]:
#from qonnx.transformation.general import ApplyConfig

### Add missing Quant nodes

In [5]:
from qonnx.transformation.quantize_graph import QuantizeGraph
# Add_4 output to be quantized, as an accumulator before ReLu (not present yet)

qnode_map = {"name": {"Add_4": [(("output",0), (scale_,zeropoint_,bitwidth_,narrow_,signed_,rounding_mode_))]}}
qnode_map

{'name': {'Add_4': [(('output', 0), (0.0009765625, 0, 18, 0, 1, 'ROUND'))]}}

In [4]:
scale_ = 0.0009765625
zeropoint_ = 0
bitwidth_ = 18
narrow_ = 0
signed_ = 1
rounding_mode_ = "ROUND"

In [6]:
quantize_graph = QuantizeGraph(qnode_map)
(qonnx_model,_) = quantize_graph.apply(qonnx_model)

In [9]:
qonnx_model.save("mod_model.onnx")